<a href="https://colab.research.google.com/github/Zero-Sik/Py_ML_Deep_Learning/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic Machine Learning from Disaster





# - Introduction 
 - This is my first work of machine learning. the Colab is written in python and has inspired from "Exploring Survival on Titanic" by Megan Risdal, a Kermel in R on kaggle

In [20]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test.csv to test (1).csv
Saving train.csv to train.csv
User uploaded file "test.csv" with length 28629 bytes
User uploaded file "train.csv" with length 61194 bytes


In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import io

train = pd.read_csv('train.csv', header= 0, dtype={'Age': np.float64})
train = pd.read_csv('test.csv', header= 0, dtype={'Age': np.float64})

full_data = [train, test]


#test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))
#train = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))




# Feature Engineering





### 1. Pclass
 - there is no missing value on this feature and already a numerical value. so let's check it's impact on our train set.
 

In [40]:
print(train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

KeyError: ignored

### 2. Sex

### SibSp and Parch
 - With the number of sibing/spouse and the number of children/parents we can create new feature called Familt Size.

### 4. Embarked
 -the embarked feature has some missing value. and we try to fill those with the most occurred value ( 'S' ).

### Fare
- Fare also has some missing value and we will replace it with the median. then we categorize it into 4 ranges.

### Age
- we have plenty of missing values in this feature. # generate random numbers between (mean - std) and (mean + std). then we categorize age into 5 range.

### Name
 - inside this feature we can find the title of people.

## Data Cleaning

## Classifier Comparison

## Prediction 